# Wrangle and Analyze Data

## Table of Contents
- [Gather](#gather)
- [Assess](#assess)
- [Clean](#clean)

In [1]:
import pandas as pd
import numpy as np
import requests
import os
import tweepy
import json
import config
import matplotlib.pyplot as plt
%matplotlib inline

<a id='gather'></a>
<h2 id="-Gather" style="
    background-color: #555;
    color: #eee;
    padding: 10px 5px;
">Gather</h2>

In [2]:
# SOURCE 1: reading the data "archive".

df_archive = pd.read_csv('twitter-archive-enhanced.csv')

In [3]:
# SOURCE 2: downloading, writeing & reading the data "predictions".

url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
response = requests.get(url)

with open(os.path.join(os.getcwd(),'image-predictions.tsv'), mode='wb') as file:
    file.write(response.content)

df_predictions = pd.read_csv('image-predictions.tsv', sep='\t')

In [4]:
# SOURCE 3: retrieving the data from twitter via APIs.

auth = tweepy.OAuthHandler(config.consumer_key, config.consumer_secret)
auth.set_access_token(config.access_token, config.access_secret)

api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [5]:
tweet_id_list = pd.concat( [df_predictions['tweet_id'], 
                          df_archive['tweet_id']], ignore_index=True)

tweet_id_list = tweet_id_list.drop_duplicates()

In [6]:
# this function just to calculate the performance 
time_prv_step = time_step = None
def performance(count,iterations):
    progress = round((count/iterations)*100,2)
    print('processing {}%'.format(progress), end='\r')
    if(count%round(iterations/10) == 0): 
        global time_prv_step
        global time_step
        
        time_prv_step = pd.Timestamp.now() if count == 0 else time_step
        time_step = pd.Timestamp.now()
        
        duration = (time_step - time_prv_step).total_seconds()
        print('{} samples, during {}s'.format(count, round(duration,2) ))
        time_start = time_step

In [7]:
tweets = []
tweet_id_list_error = []
iterations = tweet_id_list.shape[0]
count = 0
for tweet_id in tweet_id_list:
    performance(count, iterations)
    count += 1
    try:
        tweets.append(api.get_status(tweet_id)._json)
    except Exception as e:
        print(str(tweet_id) + ": " + str(e))
        tweet_id_list_error.append(tweet_id)

# df_tweets = pd.DataFrame(df_tweets)
# df_tweets.to_csv('tweet_json.txt', index=False)

0 samples, during 0.0s
236 samples, during 222.07s
472 samples, during 219.68s
680055455951884288: [{'code': 144, 'message': 'No status found with that ID.'}]
708 samples, during 242.47s
944 samples, during 266.55s
1180 samples, during 264.89s
754011816964026368: [{'code': 144, 'message': 'No status found with that ID.'}]
1416 samples, during 261.54s
802247111496568832: [{'code': 144, 'message': 'No status found with that ID.'}]
1652 samples, during 242.69s
829374341691346946: [{'code': 144, 'message': 'No status found with that ID.'}]
837012587749474308: [{'code': 144, 'message': 'No status found with that ID.'}]
837366284874571778: [{'code': 144, 'message': 'No status found with that ID.'}]
842892208864923648: [{'code': 144, 'message': 'No status found with that ID.'}]
844704788403113984: [{'code': 144, 'message': 'No status found with that ID.'}]
1888 samples, during 243.05s
851953902622658560: [{'code': 144, 'message': 'No status found with that ID.'}]
861769973181624320: [{'code':

NameError: name 'df_tweets' is not defined

In [7]:
tweets = pd.DataFrame(tweets)
tweets.to_json('tweet_json.txt', orient='records')

NameError: name 'tweets' is not defined

In [7]:
df_tweets = pd.read_json('tweet_json.txt')

> There are 22 tweets that no longer exist.

<a id='assess'></a>
<h2 id="-Gather" style="
    background-color: #555;
    color: #eee;
    padding: 10px 5px;
">Assess</h2>

<h3 id="-Gather" style="
    background-color: #bbb;
    color: #fff;
    padding: 10px 5px;
">twitter_archive_enhanced table</h3>

In [8]:
df_archive.sample(10)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
2100,670704688707301377,NaN,NaN,2015-11-28 20:43:53 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Danny. He's too good to look at the road ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/670704688...,6,10,Danny,None,None,None,None
1991,672622327801233409,NaN,NaN,2015-12-04 03:43:54 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This lil pupper is sad because we haven't foun...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/672622327...,12,10,None,None,None,pupper,None
1499,692187005137076224,NaN,NaN,2016-01-27 03:26:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is a rare Arctic Wubberfloof. Unamused by...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/692187005...,12,10,a,None,None,None,None
1170,720775346191278080,NaN,NaN,2016-04-15 00:46:48 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Berkeley. He's in a predicament. 10/10...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/720775346...,10,10,Berkeley,None,None,None,None
1523,690607260360429569,6.903413e+17,467036706.0,2016-01-22 18:49:36 +0000,"<a href=""http://twitter.com/download/iphone"" r...",12/10 @LightningHoltt,NaN,NaN,NaN,NaN,12,10,None,None,None,None,None
1478,693590843962331137,NaN,NaN,2016-01-31 00:25:18 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Phil. He's big af. Currently destroying t...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/693590843...,3,10,Phil,None,None,pupper,None
1806,676936541936185344,NaN,NaN,2015-12-16 01:27:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we see a rare pouched pupper. Ample stora...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/676936541...,8,10,None,None,None,pupper,None
1658,683111407806746624,NaN,NaN,2016-01-02 02:23:45 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Larry. He thought the New Year's parti...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/683111407...,10,10,Larry,None,None,pupper,None
2215,668542336805281792,NaN,NaN,2015-11-22 21:31:28 +0000,"<a href=""http://twitter.com/download/iphone"" r...",There's a lot going on here but in my honest o...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/668542336...,10,10,None,None,None,None,None
2122,670403879788544000,NaN,NaN,2015-11-28 00:48:35 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Nigel. He accidentally popped his ball...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/670403879...,10,10,Nigel,None,None,None,None


In [9]:
df_archive.describe()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,retweeted_status_id,retweeted_status_user_id,rating_numerator,rating_denominator
count,2.356000e+03,7.800000e+01,7.800000e+01,1.810000e+02,1.810000e+02,2356.000000,2356.000000
mean,7.427716e+17,7.455079e+17,2.014171e+16,7.720400e+17,1.241698e+16,13.126486,10.455433
std,6.856705e+16,7.582492e+16,1.252797e+17,6.236928e+16,9.599254e+16,45.876648,6.745237
min,6.660209e+17,6.658147e+17,1.185634e+07,6.661041e+17,7.832140e+05,0.000000,0.000000
25%,6.783989e+17,6.757419e+17,3.086374e+08,7.186315e+17,4.196984e+09,10.000000,10.000000
50%,7.196279e+17,7.038708e+17,4.196984e+09,7.804657e+17,4.196984e+09,11.000000,10.000000
75%,7.993373e+17,8.257804e+17,4.196984e+09,8.203146e+17,4.196984e+09,12.000000,10.000000
max,8.924206e+17,8.862664e+17,8.405479e+17,8.874740e+17,7.874618e+17,1776.000000,170.000000


In [10]:
df_archive.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

In [11]:
# this function to explore counts of categorical values in each column of provided df.
def categorical_scan(df):
    for col in df.columns:
        print(df[col].value_counts().count(), '\t', col)

In [12]:
categorical_scan(df_archive)

2356 	 tweet_id
77 	 in_reply_to_status_id
31 	 in_reply_to_user_id
2356 	 timestamp
4 	 source
2356 	 text
181 	 retweeted_status_id
25 	 retweeted_status_user_id
181 	 retweeted_status_timestamp
2218 	 expanded_urls
40 	 rating_numerator
18 	 rating_denominator
957 	 name
2 	 doggo
2 	 floofer
2 	 pupper
2 	 puppo


In [13]:
df_archive.rating_denominator.value_counts()

10     2333
11        3
50        3
80        2
20        2
2         1
16        1
40        1
70        1
15        1
90        1
110       1
120       1
130       1
150       1
170       1
7         1
0         1
Name: rating_denominator, dtype: int64

In [14]:
df_archive.text.str.contains(r'\d+\/\d+').value_counts()

True    2356
Name: text, dtype: int64

In [15]:
df_archive.text.str.extract(r'(?P<text>\d+\/\d+)').text.value_counts()

12/10      558
11/10      463
10/10      461
13/10      351
9/10       156
8/10       102
14/10       54
7/10        53
5/10        37
6/10        32
3/10        19
4/10        15
2/10         9
1/10         8
420/10       2
75/10        2
9/11         2
4/20         2
0/10         2
15/10        2
7/11         1
88/80        1
99/90        1
17/10        1
44/40        1
165/150      1
182/10       1
1776/10      1
143/130      1
666/10       1
80/80        1
007/10       1
27/10        1
960/00       1
20/16        1
50/50        1
45/50        1
60/50        1
204/170      1
121/110      1
144/120      1
24/7         1
84/70        1
26/10        1
11/15        1
1/2          1
Name: text, dtype: int64

In [16]:
# retweets in tweets
df_archive[
    df_archive.retweeted_status_id.isin(df_archive.tweet_id)
].shape[0]


156

In [17]:
# replies in tweets
df_archive[
    df_archive.in_reply_to_status_id.isin(df_archive.tweet_id)
].shape[0]

44

<h3 id="-Gather" style="
    background-color: #bbb;
    color: #fff;
    padding: 10px 5px;
">image_predictions table</h3>

In [18]:
df_predictions.sample(20)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
1215,743510151680958465,https://pbs.twimg.com/ext_tw_video_thumb/74350...,1,sea_lion,0.859046,False,tub,0.020405,False,hippopotamus,0.013095,False
1203,741438259667034112,https://pbs.twimg.com/media/CkoeKTPWYAAcWmo.jpg,1,Chesapeake_Bay_retriever,0.292675,True,redbone,0.197858,True,vizsla,0.150312,True
2064,890006608113172480,https://pbs.twimg.com/media/DFnwSY4WAAAMliS.jpg,1,Samoyed,0.957979,True,Pomeranian,0.013884,True,chow,0.008167,True
576,678675843183484930,https://pbs.twimg.com/media/CWskEqnWUAAQZW_.jpg,1,maze,0.339850,False,streetcar,0.099688,False,sundial,0.084808,False
1312,754482103782404096,https://pbs.twimg.com/ext_tw_video_thumb/75448...,1,tub,0.596796,False,bathtub,0.381098,False,shower_curtain,0.017629,False
1340,758740312047005698,https://pbs.twimg.com/media/CoeWSJcUIAAv3Bq.jpg,1,Chesapeake_Bay_retriever,0.848514,True,Labrador_retriever,0.110054,True,curly-coated_retriever,0.025201,True
21,666293911632134144,https://pbs.twimg.com/media/CT8mx7KW4AEQu8N.jpg,1,three-toed_sloth,0.914671,False,otter,0.015250,False,great_grey_owl,0.013207,False
1511,786233965241827333,https://pbs.twimg.com/media/CulDnZpWcAAGbZ-.jpg,1,Labrador_retriever,0.478193,True,schipperke,0.224817,True,Staffordshire_bullterrier,0.077396,True
1031,711363825979756544,https://pbs.twimg.com/media/Cd9Fn5QUMAAYMT4.jpg,1,Pembroke,0.750906,True,Cardigan,0.241152,True,basenji,0.002640,True
997,708356463048204288,https://pbs.twimg.com/media/CdSWcc1XIAAXc6H.jpg,2,pug,0.871283,True,French_bulldog,0.041820,True,bath_towel,0.015228,False


In [19]:
df_predictions.describe()

,tweet_id,img_num,p1_conf,p2_conf,p3_conf
count,2.075000e+03,2075.000000,2075.000000,2.075000e+03,2.075000e+03
mean,7.384514e+17,1.203855,0.594548,1.345886e-01,6.032417e-02
std,6.785203e+16,0.561875,0.271174,1.006657e-01,5.090593e-02
min,6.660209e+17,1.000000,0.044333,1.011300e-08,1.740170e-10
25%,6.764835e+17,1.000000,0.364412,5.388625e-02,1.622240e-02
50%,7.119988e+17,1.000000,0.588230,1.181810e-01,4.944380e-02
75%,7.932034e+17,1.000000,0.843855,1.955655e-01,9.180755e-02
max,8.924206e+17,4.000000,1.000000,4.880140e-01,2.734190e-01


In [20]:
df_predictions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [21]:
categorical_scan(df_predictions)

2075 	 tweet_id
2009 	 jpg_url
4 	 img_num
378 	 p1
2006 	 p1_conf
2 	 p1_dog
405 	 p2
2004 	 p2_conf
2 	 p2_dog
408 	 p3
2006 	 p3_conf
2 	 p3_dog


In [22]:
df_predictions.p1.value_counts()

golden_retriever             150
Labrador_retriever           100
Pembroke                      89
Chihuahua                     83
pug                           57
chow                          44
Samoyed                       43
toy_poodle                    39
Pomeranian                    38
malamute                      30
cocker_spaniel                30
French_bulldog                26
Chesapeake_Bay_retriever      23
miniature_pinscher            23
seat_belt                     22
Siberian_husky                20
Staffordshire_bullterrier     20
German_shepherd               20
Cardigan                      19
web_site                      19
beagle                        18
Shetland_sheepdog             18
Eskimo_dog                    18
Maltese_dog                   18
teddy                         18
Rottweiler                    17
Lakeland_terrier              17
Shih-Tzu                      17
kuvasz                        16
Italian_greyhound             16
          

In [23]:
df_predictions.p2.value_counts()

Labrador_retriever                104
golden_retriever                   92
Cardigan                           73
Chihuahua                          44
Pomeranian                         42
French_bulldog                     41
Chesapeake_Bay_retriever           41
toy_poodle                         37
cocker_spaniel                     34
Siberian_husky                     33
miniature_poodle                   33
beagle                             28
Pembroke                           27
collie                             27
Eskimo_dog                         27
kuvasz                             26
Italian_greyhound                  22
Pekinese                           21
American_Staffordshire_terrier     21
chow                               20
toy_terrier                        20
malinois                           20
miniature_pinscher                 20
Samoyed                            20
Norwegian_elkhound                 19
Boston_bull                        19
Staffordshir

In [24]:
df_predictions.p3.value_counts()

Labrador_retriever                79
Chihuahua                         58
golden_retriever                  48
Eskimo_dog                        38
kelpie                            35
kuvasz                            34
chow                              32
Staffordshire_bullterrier         32
cocker_spaniel                    31
beagle                            31
Pomeranian                        29
toy_poodle                        29
Pekinese                          29
Chesapeake_Bay_retriever          27
Pembroke                          27
Great_Pyrenees                    27
French_bulldog                    26
malamute                          26
American_Staffordshire_terrier    24
pug                               23
Cardigan                          23
basenji                           21
bull_mastiff                      20
toy_terrier                       20
Siberian_husky                    19
Shetland_sheepdog                 17
Boston_bull                       17
b

In [25]:
df_predictions.query('p1_dog == False & p2_dog == False & p3_dog == False').shape[0]

324

<h3 id="-Gather" style="
    background-color: #bbb;
    color: #fff;
    padding: 10px 5px;
">tweets table</h3>

In [26]:
df_tweets.sample(10)

,contributors,coordinates,created_at,entities,extended_entities,favorite_count,favorited,geo,id,id_str,...,quoted_status,quoted_status_id,quoted_status_id_str,retweet_count,retweeted,retweeted_status,source,text,truncated,user
1716,NaN,NaN,2017-01-11 17:01:16,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 819227674182828033, 'id_str'...",24136,False,NaN,819227688460238848,819227688460238848,...,None,NaN,NaN,7117,False,None,"<a href=""http://twitter.com/download/iphone"" r...",This is Finn. He's wondering if you come here ...,False,"{'id': 4196983835, 'id_str': '4196983835', 'na..."
1348,NaN,NaN,2016-07-31 16:50:42,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 759793413751472128, 'id_str'...",6195,False,NaN,759793422261743616,759793422261743616,...,None,NaN,NaN,2001,False,None,"<a href=""http://twitter.com/download/iphone"" r...","Meet Maggie &amp; Lila. Maggie is the doggo, L...",False,"{'id': 4196983835, 'id_str': '4196983835', 'na..."
2226,NaN,NaN,2016-07-09 15:02:31,"{'hashtags': [], 'symbols': [], 'user_mentions...",None,6068,False,NaN,751793661361422336,751793661361422336,...,None,NaN,NaN,3037,False,None,"<a href=""http://vine.co"" rel=""nofollow"">Vine -...",This is Fred. He's having one heck of a summer...,False,"{'id': 4196983835, 'id_str': '4196983835', 'na..."
865,NaN,NaN,2016-02-10 18:11:03,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 697482920076951552, 'id_str'...",2484,False,NaN,697482927769255936,697482927769255936,...,None,NaN,NaN,696,False,None,"<a href=""http://twitter.com/download/iphone"" r...",Meet Blipson. He's a Doowap Hufflepuff. That U...,False,"{'id': 4196983835, 'id_str': '4196983835', 'na..."
2137,NaN,NaN,2017-01-24 02:28:08,"{'hashtags': [], 'symbols': [], 'user_mentions...",None,0,False,NaN,823719002937630720,823719002937630720,...,None,NaN,NaN,11851,False,{'created_at': 'Thu Oct 06 15:49:14 +0000 2016...,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This is Balto. He's very conten...,False,"{'id': 4196983835, 'id_str': '4196983835', 'na..."
1249,NaN,NaN,2016-06-28 20:14:22,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 747885821378715648, 'id_str'...",3046,False,NaN,747885874273214464,747885874273214464,...,None,NaN,NaN,1026,False,None,"<a href=""http://twitter.com/download/iphone"" r...",This is a mighty rare blue-tailed hammer sherk...,False,"{'id': 4196983835, 'id_str': '4196983835', 'na..."
1907,NaN,NaN,2017-04-18 23:50:52,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 854482381390073856, 'id_str'...",29367,False,NaN,854482394044301312,854482394044301312,...,None,NaN,NaN,6926,False,None,"<a href=""http://twitter.com/download/iphone"" r...",This is Arya. She can barely contain her excit...,False,"{'id': 4196983835, 'id_str': '4196983835', 'na..."
1992,NaN,NaN,2017-06-21 00:06:44,"{'hashtags': [], 'symbols': [], 'user_mentions...",None,26513,False,NaN,877316821321428993,877316821321428992,...,None,NaN,NaN,4961,False,None,"<a href=""http://twitter.com/download/iphone"" r...",Meet Dante. At first he wasn't a fan of his ne...,True,"{'id': 4196983835, 'id_str': '4196983835', 'na..."
1268,NaN,NaN,2016-07-04 01:17:51,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 749774184847446016, 'id_str'...",4820,False,NaN,749774190421639168,749774190421639168,...,None,NaN,NaN,1372,False,None,"<a href=""http://twitter.com/download/iphone"" r...",This is Lucy. She's a Benebop Cumberplop. 12/1...,False,"{'id': 4196983835, 'id_str': '4196983835', 'na..."
1430,NaN,NaN,2016-09-06 23:56:05,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 773308813344645120, 'id_str'...",24303,False,NaN,773308824254029826,773308824254029824,...,None,NaN,NaN,7892,False,None,"<a href=""http://twitter.com/download/iphone"" r...",This is Bungalo. She uses that face to get wha...,False,"{'id': 4196983835, 'id_str': '4196983835', 'na..."


In [27]:
df_tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2334 entries, 0 to 2333
Data columns (total 30 columns):
contributors                     0 non-null float64
coordinates                      0 non-null float64
created_at                       2334 non-null datetime64[ns]
entities                         2334 non-null object
extended_entities                1819 non-null object
favorite_count                   2334 non-null int64
favorited                        2334 non-null bool
geo                              0 non-null float64
id                               2334 non-null int64
id_str                           2334 non-null int64
in_reply_to_screen_name          77 non-null object
in_reply_to_status_id            77 non-null float64
in_reply_to_status_id_str        77 non-null float64
in_reply_to_user_id              77 non-null float64
in_reply_to_user_id_str          77 non-null float64
is_quote_status                  2334 non-null bool
lang                             2334 no

In [28]:
categorical_scan(df_tweets)

0 	 contributors
0 	 coordinates
2334 	 created_at
2241 	 entities
1819 	 extended_entities
1995 	 favorite_count
1 	 favorited
0 	 geo
2334 	 id
2334 	 id_str
30 	 in_reply_to_screen_name
76 	 in_reply_to_status_id
76 	 in_reply_to_status_id_str
30 	 in_reply_to_user_id
30 	 in_reply_to_user_id_str
2 	 is_quote_status
9 	 lang
1 	 place
1 	 possibly_sensitive
1 	 possibly_sensitive_appealable
24 	 quoted_status
26 	 quoted_status_id
26 	 quoted_status_id_str
1733 	 retweet_count
1 	 retweeted
165 	 retweeted_status
4 	 source
2334 	 text
2 	 truncated
250 	 user


### Quality
#### `twitter_archive_enhanced` table
- There are "None" as value needs to be corrected.
- There are number of observations have `doggo` with others.
- The `tweet_id` need to be string.
- There are 2 columns, end with `timestamp`, are object type while they need to be time formate.
- `source` column has values in HTML fromat.
- `rating_denominator` has values differ 10.
- `rating_numerator` has some mistakes (not extracted correctly from `text`).
- `rating_numerator` has anomalous values >= 45.9 (std) which we can consider them as outliers.
- There are 156 tweets as retweets.
- There are unwanted columns of `twitter_archive_enhanced` should be removed.

#### `image_predictions` table
- `id` column should be `string` type.
- There are 324 observations are not predected as a dog.
- There are unwanted columns of `image_predictions` should be removed.

#### `tweets` table
- `id` column should be `string` type.
- There are columns should be droped becuase either they have 1 repeated value or nothing.
- There are unwanted columns of `tweets` should be removed.

### Tidiness
- `doggo`, `floofer`, `pupper` & `puppo` columns of `twitter_archive_enhanced` have to be values in the new column called "type".
- join all tables to create `twitter_archive_master.csv`.

<a id='clean'></a>
<h2 id="-Gather" style="
    background-color: #555;
    color: #eee;
    padding: 10px 5px;
">Clean</h2>

In [218]:
df2_archive = df_archive.copy()
df2_predictions = df_predictions.copy()
df2_tweets = df_tweets.copy()

<h3 id="-Gather" style="
    background-color: #bbb;
    color: #fff;
    padding: 10px 5px;
">twitter_archive_enhanced table</h3>

## There are "None" as value needs to be corrected.

### Define

> change "None" value to be np.nan

### Code

In [219]:
df2_archive = df2_archive.applymap(lambda x: np.nan if x == 'None' else x)

### Test

In [220]:
# if there is a different of counts, means it is correct
(df_archive.count() == df2_archive.count()).all() != True

True

## There are number of observations have `doggo` with others.

### Define

> merge `doggoe` with others into `multiple` column.

### Code

In [221]:
df2_archive['multiple'] = df2_archive[['doggo','floofer','pupper','puppo']].apply(
    lambda x: ','.join(x.dropna().astype('str')), axis=1
)
df2_archive.multiple = df2_archive.multiple.map(lambda x: np.nan if len(x)==0 else x)

### Test

In [222]:
df2_archive.multiple.value_counts()

pupper           245
doggo             83
puppo             29
doggo,pupper      12
floofer            9
doggo,puppo        1
doggo,floofer      1
Name: multiple, dtype: int64

## The `tweet_id` need to be string.

### Define

> change the type of columns end with `_id` to string.

### Code

In [223]:
df2_archive.tweet_id = df2_archive.tweet_id.astype('str')

### Test

In [224]:
df2_archive.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 18 columns):
tweet_id                      2356 non-null object
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          1611 non-null object
doggo                         97 non-null object
floofer                       10 non-null object
pupper                        257 non-null object
puppo                         30 non-null object
multiple                      380 non-

## There are 2 columns, end with `timestamp`, are object type while they need to be `timestamp`.

### Define

> change the type of columns end with `timestamp` from object to timestamp. 

### Code

In [225]:
for col in df2_archive.columns[df2_archive.columns.str.endswith('timestamp')]:
    df2_archive[col] = pd.to_datetime(df2_archive[col])

### Test

In [226]:
df2_archive.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 18 columns):
tweet_id                      2356 non-null object
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null datetime64[ns, UTC]
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null datetime64[ns, UTC]
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          1611 non-null object
doggo                         97 non-null object
floofer                       10 non-null object
pupper                        257 non-null object
puppo                         30 non-null object
multiple    

## `source` column has values in HTML fromat.

### Define

> correct the value by remvoing HMTL tag

### Code

In [227]:
df2_archive['source'] = df2_archive.source.str.replace('<[^>]*>','')

### Test

In [228]:
df2_archive.source.value_counts()

Twitter for iPhone     2221
Vine - Make a Scene      91
Twitter Web Client       33
TweetDeck                11
Name: source, dtype: int64

## `rating_denominator` has values differ 10.

### Define

> remove all observations have `rating_denominator` differ 10.

### Code

In [229]:
df2_archive.drop(df2_archive[
    df2_archive.rating_denominator != 10
].index, inplace=True)

### Test

In [230]:
# the lengths should be difference
df2_archive.shape[0] != df_archive.shape[0]

True

## `rating_numerator` has some mistakes (not extracted correctly from `text`).

### Define

> replace boservations has mistaken by correct one from `text`.

### Code

In [231]:
rating = df2_archive.text.str.extract('((?:\d+\.)?\d+)\/(\d+)', expand=True)
rating.columns = ['rating_numerator', 'rating_denominator']
rating.rating_denominator = rating.rating_denominator.astype('float64')
rating.rating_numerator = rating.rating_numerator.astype('float64')

df2_archive.loc[
    df2_archive.rating_numerator != rating.rating_numerator, 'rating_numerator'
] = rating.loc[
    df2_archive.rating_numerator != rating.rating_numerator, 'rating_numerator'
]

### Test

In [232]:
(df2_archive.rating_numerator.astype('str').str.extract('(\d+)\.(\d+)')[1].astype('int64') > 0).any()

True

## `rating_numerator` has anomalous values >= 50 which are outliers.

### Define

> remvoe all observations have `rating_numerator` >= 45.9 (std) which we can consider them as outliers.

### Code

In [233]:
df2_archive.drop(
    df2_archive[df2_archive.rating_numerator >= 50].index
, inplace=True)

### Test

In [234]:
df2_archive.groupby('rating_numerator').rating_numerator.count()

rating_numerator
0.00       2
1.00       8
2.00       9
3.00      19
4.00      15
5.00      35
6.00      32
7.00      54
8.00     102
9.00     156
9.50       1
9.75       2
10.00    461
11.00    463
11.26      1
11.27      1
12.00    558
13.00    351
13.50      1
14.00     54
15.00      2
17.00      1
Name: rating_numerator, dtype: int64

## There are 156 tweets as retweets.

### Define

> remove all observations which are actually retweets.

### Code

In [235]:
df2_archive.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo,multiple
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56+00:00,Twitter for iPhone,This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaT,https://twitter.com/dog_rates/status/892420643...,13.0,10,Phineas,NaN,NaN,NaN,NaN,NaN
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27+00:00,Twitter for iPhone,This is Tilly. She's just checking pup on you....,NaN,NaN,NaT,https://twitter.com/dog_rates/status/892177421...,13.0,10,Tilly,NaN,NaN,NaN,NaN,NaN
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03+00:00,Twitter for iPhone,This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaT,https://twitter.com/dog_rates/status/891815181...,12.0,10,Archie,NaN,NaN,NaN,NaN,NaN
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51+00:00,Twitter for iPhone,This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaT,https://twitter.com/dog_rates/status/891689557...,13.0,10,Darla,NaN,NaN,NaN,NaN,NaN
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24+00:00,Twitter for iPhone,This is Franklin. He would like you to stop ca...,NaN,NaN,NaT,https://twitter.com/dog_rates/status/891327558...,12.0,10,Franklin,NaN,NaN,NaN,NaN,NaN


In [236]:
df2_archive.drop(
    df2_archive[
        df2_archive.retweeted_status_id.isin(df2_archive.tweet_id)
    ].index,
    inplace=True
)

### Test

In [237]:
df2_archive.shape[0] != df_archive.shape[0]

True

## There are unwanted columns of `twitter_archive_enhanced` should be removed.

### Define

> rmove `in_reply_to_status_id`, `in_reply_to_user_id`, `retweeted_status_id`, `retweeted_status_user_id`, `retweeted_status_timestamp`, `expanded_urls` & `text` columns.

### Code

In [238]:
df2_archive.drop(
    ['in_reply_to_status_id',
    'in_reply_to_user_id',
    'retweeted_status_id',
    'retweeted_status_user_id',
    'retweeted_status_timestamp',
    'expanded_urls',
    'text'], axis=1, inplace=True
)

### Test

In [239]:
list(df2_archive)

['tweet_id',
 'timestamp',
 'source',
 'rating_numerator',
 'rating_denominator',
 'name',
 'doggo',
 'floofer',
 'pupper',
 'puppo',
 'multiple']

<h3 id="-Gather" style="
    background-color: #bbb;
    color: #fff;
    padding: 10px 5px;
">image_predictions table</h3>

## `id` column should be `string` type.

### Define

> change the type of `tweet_id` to be string.

### Code

In [240]:
df2_predictions.tweet_id = df2_predictions.tweet_id.astype('str')

### Test

In [241]:
df2_predictions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null object
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(1), object(5)
memory usage: 152.1+ KB


## there are 324 observations are not predected as a dog.

### Define

> remove all observations were not predected as a dog.

### Code

In [242]:
df2_predictions.drop(
    df2_predictions.query('p1_dog == False & p2_dog == False & p3_dog == False').index
    , inplace=True
)

### Test

In [243]:
# they should be not equal
df2_predictions.shape[0] != df_predictions.shape[0]

True

## There are unwanted columns of `image_predictions` should be removed.

### Define

> remvoe `img_num` columns.

### Code

In [244]:
df2_predictions.drop(['img_num'], axis=1, inplace=True)

### Test

In [245]:
list(df2_predictions)

['tweet_id',
 'jpg_url',
 'p1',
 'p1_conf',
 'p1_dog',
 'p2',
 'p2_conf',
 'p2_dog',
 'p3',
 'p3_conf',
 'p3_dog']

<h3 id="-Gather" style="
    background-color: #bbb;
    color: #fff;
    padding: 10px 5px;
">twitter table</h3>

## `id` column should be `string` type.

### Define

> change the type of `id` to be string.

### Code

In [246]:
df2_tweets.id = df2_tweets.id.astype('str')

### Test

In [247]:
df2_tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2334 entries, 0 to 2333
Data columns (total 30 columns):
contributors                     0 non-null float64
coordinates                      0 non-null float64
created_at                       2334 non-null datetime64[ns]
entities                         2334 non-null object
extended_entities                1819 non-null object
favorite_count                   2334 non-null int64
favorited                        2334 non-null bool
geo                              0 non-null float64
id                               2334 non-null object
id_str                           2334 non-null int64
in_reply_to_screen_name          77 non-null object
in_reply_to_status_id            77 non-null float64
in_reply_to_status_id_str        77 non-null float64
in_reply_to_user_id              77 non-null float64
in_reply_to_user_id_str          77 non-null float64
is_quote_status                  2334 non-null bool
lang                             2334 n

## There are columns should be droped becuase either they have 1 repeated value or nothing.

### Define

> remove all columns that have 1 repeatted value or nothing.

### Code

In [248]:
cols = []
for col in df2_tweets.columns:
    if(df2_tweets[col].value_counts().count() <=1):
        cols.append(col)


In [249]:
df2_tweets.drop(cols, axis=1, inplace=True)

### Test

In [250]:
for col in df2_tweets.columns:
    print(df2_tweets[col].value_counts().count(), '\t', col)

2334 	 created_at
2241 	 entities
1819 	 extended_entities
1995 	 favorite_count
2334 	 id
2334 	 id_str
30 	 in_reply_to_screen_name
76 	 in_reply_to_status_id
76 	 in_reply_to_status_id_str
30 	 in_reply_to_user_id
30 	 in_reply_to_user_id_str
2 	 is_quote_status
9 	 lang
24 	 quoted_status
26 	 quoted_status_id
26 	 quoted_status_id_str
1733 	 retweet_count
165 	 retweeted_status
4 	 source
2334 	 text
2 	 truncated
250 	 user


## There are unwanted columns of `tweets` should be removed.

### Define

> remvoe `id`, `favorite_count` & `retweet_count` columns.

### Code

In [251]:
df2_tweets.drop(
    df2_tweets.columns.difference(['id', 'favorite_count', 'retweet_count']), axis=1, inplace=True
)

### Test

In [252]:
list(df2_tweets)

['favorite_count', 'id', 'retweet_count']

<h3 id="-Gather" style="
    background-color: #bbb;
    color: #fff;
    padding: 10px 5px;
">Tidiness</h3>

In [253]:
df3_predictions = df2_predictions.copy()
df3_archive = df2_archive.copy()
df3_tweets = df2_tweets.copy()

## `doggo`, `floofer`, `pupper` & `puppo` columns of "twitter_archive_enhanced" have to be values in the new column called `type`.


In [254]:
value_vars = ['multiple']
cols = df3_archive.columns.difference(value_vars)

df3_archive = df3_archive.melt(
    id_vars=cols, value_vars=value_vars, value_name='stage'
)

df3_archive.reset_index(inplace=True)
df3_archive.drop('variable', axis=1, inplace=True)

In [255]:
df3_archive.stage.value_counts()

pupper           228
doggo             75
puppo             24
doggo,pupper      10
floofer            9
doggo,puppo        1
doggo,floofer      1
Name: stage, dtype: int64

## join all tables to create `twitter_archive_master.csv`.

In [256]:
df3_tweets.rename({'id':'tweet_id'}, axis=1, inplace=True)

In [257]:
# without reset index, the merge will not working properly

df3_master = df3_archive.merge(
    df3_predictions, how='outer', on='tweet_id'
).merge(
    df3_tweets, how='outer', on='tweet_id'
)

In [258]:
df3_master.to_csv('twitter_archive_master.csv', index=False)